In [1]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)

import torch

In [2]:
compute_dtype = getattr(torch, "float16")
#모델을 4bit 형식으로 로드(메모리 소비가 줄어듦)
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, 
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/gatoai/python/venv/3.10/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# tokenizer_ko = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b",trust_remote_code=True,padding_side="right", padding=True,add_eos_token=True,add_bos_token=True,use_fast=False)
#     #tokenizer.pad_token = tokenizer.eos_token: 이 부분은 패딩 토큰을 EOS (End-Of-Sequence) 토큰으로 설정합니다. 이렇게 하면 모델이 패딩을 식별하는 데 사용되는 토큰을 EOS 토큰으로 사용하게 됩니다.
# tokenizer_ko.pad_token = tokenizer_ko.eos_token

In [5]:
from peft import PeftModel
# is_trainable=False여기서는 이 PEFT 모델로만 추론을 수행할 계획이므로 설정
ft_model = PeftModel.from_pretrained(base_model, "./model/peft-ko-training-1712189085/checkpoint-1000/",torch_dtype=torch.float16,is_trainable=False)

#미세 조정은 종종 반복적인 프로세스입니다. 
#검증 및 테스트 세트 결과에 따라 성능을 향상시키기 위해 
#모델의 아키텍처, 하이퍼파라미터 또는 교육 데이터를 추가로 조정해야 할 수도 있습니다. 

In [6]:
huggingface_dataset_name = "beomi/KoAlpaca-v1.1a"
#data set load
dataset = load_dataset(huggingface_dataset_name)

In [7]:
def tokenizer_fun(tokenizer_name):
    '''
        토크나이저를 설정하는 함수
    '''
    tokenizer_ko = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b",trust_remote_code=True,padding_side="right", padding=True,add_eos_token=True,add_bos_token=True,use_fast=False)
    #tokenizer.pad_token = tokenizer.eos_token: 이 부분은 패딩 토큰을 EOS (End-Of-Sequence) 토큰으로 설정합니다. 이렇게 하면 모델이 패딩을 식별하는 데 사용되는 토큰을 EOS 토큰으로 사용하게 됩니다.
    tokenizer_ko.pad_token = tokenizer_ko.eos_token

    return tokenizer_ko

tokenizer_ko = tokenizer_fun('EleutherAI/polyglot-ko-5.8b')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
def gen(model_name, prompt, max_length):
    # 입력 텍스트를 토크나이즈하고 모델에 입력할 형식으로 변환
    input_ids = tokenizer_ko.encode(prompt, max_length=max_length, padding=True, return_tensors="pt", truncation=True, return_attention_mask=True)
    
    # 모델에 입력하여 텍스트 생성
    outputs = model_name.generate(input_ids.to('cuda'), max_length=max_length,pad_token_id=tokenizer_ko.eos_token_id)

    # print(len(outputs))
    # print(outputs)
    
    
    # 생성된 텍스트 디코딩
    generated_text = tokenizer_ko.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

In [29]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters 
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Answer the questions below"
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"
    
    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['instruction']}" if sample["instruction"] else None
    response = f"{RESPONSE_KEY}\n"
    # end = f"{END_KEY}"
    
    parts = [part for part in [blurb, instruction, input_context, response] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

In [37]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42
dash_line = '-'.join('' for x in range(100))


set_seed(seed)

index = 20000
sample = dataset['train'][index]
# sample = {"instruction": '애국가 불러줘'}
print(sample)
print(dash_line)
prompt = create_prompt_formats(sample)

print(prompt['text'])

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt['text'],550)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(dash_line)
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

print(dash_line)
print(f'INPUT PROMPT:\n{prompt["instruction"]}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{prompt["output"]}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

{'instruction': "'땡추가 뭐예요?'라는 말이 무슨 뜻인가요? 이 말의 유래와 역사적인 배경을 알려주세요.", 'output': "'땡추'는 원래 고려시대에 개혁을 추진한 몽골의 지배에 벗어나고자 노력한 스님들의 모임입니다. 그러나 조선시대 불교 탄압으로 인해 '땡추'라는 단어는 부정적인 의미로 바뀌었으며, '땡중'이라는 단어로 파생되어 술과 고기를 즐기고 수행에 관심이 없는 스님들을 비아냥하는 뜻으로 쓰이기도 합니다. 땡추는 전국적으로 조직화 되어 있었고, 부당한 대우나 형편에 처한 경우 서로 돕고 오랜 조직원 가운데 어떤 사람이 저지른 실수나 곤란한 일을 당하면 반드시 복수하는 등 자체 법집을 갖추고 있었습니다. 이러한 역사적인 배경을 갖고 있어, '땡추가 뭐예요?'라고 하는 것은 불친절한 말이기 때문에 조심해야 합니다.", 'url': 'https://kin.naver.com/qna/detail.naver?d1id=6&dirId=613&docId=61653563'}
---------------------------------------------------------------------------------------------------

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruct: Answer the questions below

'땡추가 뭐예요?'라는 말이 무슨 뜻인가요? 이 말의 유래와 역사적인 배경을 알려주세요.

### Output:

---------------------------------------------------------------------------------------------------
'땡추가 뭐예요?'라는 말이 무슨 뜻은 말의 유래와 역사적인 배경을 알려주세요. 이 말의 유래와 역사적인 배경을 알려주세요. 이 말의 유래와 역사적인 배

In [23]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 1
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n{instruction}\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 미치ruction' :
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction:
Out:
 미치ruction
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
스웨터의 유래는 어디에서 시작되었나요?
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 시작되었습니다. 노동자들의 방한복에서 시작된 스웨터는 여가생활과 스포츠의 붐에 힘입어 대중화되

In [24]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 0
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n너의 이름은 뭐야?\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below 그냥ruction:Below 그냥ruction' "'se'i 이름:
Out:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
Below 그냥ruction:
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
너의 이름은 뭐야?
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌

In [25]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 100
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n{instruction}\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


충전기에 꼽다 아니면 꽂다깔 어느 것을 사용 하나요? 급합니다! (내공 90:

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
충전기에 꼽다 아니면 꽂다 중 어느 것을 사용해야 하나요? 급합니다! (내공 90)
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
꼽다와 꽂다는 많은 사람들이 헷갈려하는 단어입니다. 하지만 이 둘을 구분하는 방법이 있습니다. "꼽다"는 수나 날짜 등을 세려고 손가락을 헤아릴 때 사용하고, "꽂다"는 쓰러지지 않게 세우거나 고정할 때 사용합니다. 따라서 충전기를 콘센트에 연결할 때는 "꽂다"라는 표현을 사용하면 됩니다.

---------------------------------------------------------------------------------------------------
PEFT MODEL:
충전기에 꼽다 아니면 꽂다깔 어느 것을 사용 하나요? 급합니다! (내공 90:

CPU times: user 23.9 s, sys: 72.3 ms, total: 24 s
Wall time: 24.1 s


In [26]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 300
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n{instruction}\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


피 늘 감elbelow 그냥below 그냥belelow 그냥below 그냥belelow 그냥belelow 그냥belelow 그냥belelow 그냥belelow 그냥belelow 그냥belelow 그냥belelow 그냥belelow 그냥belelow 그냥belelow 그냥belelow 그냥belelelow 그냥belelelow 그냥belelelow 그냥belelelow 그냥belelelow 그냥belelelow 그냥belelelow 그냥belelelow 그냥belelelow 그냥belelelow 그냥belelelelow 그냥belelelow 그냥belelelelow 그냥belelelow 그냥belelelow 그냥belelelow 그냥belelelow 그냥belelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelel
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
피그말리온 효과란 무엇인가요? 구체적인 예시를 들어 설명해주세요.
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
피그말리온 효과는 교사나 학부모 등 다른 사람의 높은 기대나 기대에 부응하기 위해 노력하는 개인이 그 기대를 충족시키는 현

In [27]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 1666
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n{instruction}\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Uow 그냥belelbelelbelow 그냥belelbelow 그냥belelbelelbelow 그냥belelbelelow 그냥belelelbelow 그냥belelelow 그냥belelelbelow 그냥belelelelow 그냥belelelelow 그냥belelelow 그냥belelelow 그냥belelelow 그냥belelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelelel
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
왜 utp선은 꼬여 있을까요?
UTP선 보면 꼬여 있는데 그 이유가 무엇인가요?
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
UTP케이블을 제작할 때, 케이블의 양쪽 끝의 배열이 동일하지 않으면 1~2미터 짧은 거리에서는 작동할 수 있지만, 전송률을 높이고 외부 전기 노이즈

In [28]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 21000
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n{instruction}\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


elelowb 그냥 자랑:elowb 그냥 자랑' struction:
클렌징폼과 비누의 차이가 무엇인가요?
#### Instruction:
클렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼과 비누의 차이가 무엇인가요?
렌징폼
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
클렌징폼과 비누의 차이가 무엇인가요?
얼굴 세정을 위해 클렌징폼과 비누를 모두 사용할 수 있지만, 클렌징폼과 비누의 차이가 궁금합니다.
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
클렌징폼과 일반 비누의 가장 큰 차이는 세정력입니다. 일반비누보다는 클렌징폼의 세정력이 더 낫습니다. 하지만 일반 비누는 얼굴을 씻는 용도로 사용하면 안되며 반드시 클렌징폼으로 세정해주셔야 합니다.

일반 비누는 PH 9~10인 알칼리성이기 때문에 우리의 피부 PH인 4.5~6.5와 차이가

In [29]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 1111
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n{instruction}\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


빛의 속도가 30만kmelb 그냥elb 그냥elrelb 그냥elrelb 그냥elrelb 그냥elrelb 그냥elrelb 그냥elrelb 그냥elrelb 그냥elrelb 그냥elrelrelb 그냥elrelrelrelb 그냥elrelrelrelb 그냥elrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelrelr
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
빛의 속도가 30만km인 것을 어떻게 알았을까요? 과학적 증명이나 실험에 의해 된 것이라면 근거를 알고 싶습니다.
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
빛의 속도는 실험과 이론 양쪽으로부터 아주 정밀히 구할 수 있습니다. 1676년에는 덴마크의 뢰머라는 학자가 목성의 위성인 이오가 목성에 가려지는 현상을 이용하여 빛의 속도를 약 22만 km/s로 산출했습니다. 이후 브래들리의 광행차를 이용한 계산, 톱니바퀴를 이용한 피조의 방법, 그리고 19세기 후반에는 간섭계 등이 발견되어 정밀측정이 가능해졌습니다.

In [30]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 577
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n{instruction}\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


elowruction' :
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를struction:
서울 지하철 노선를
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
서울 지하철 노선별 속도는 어떻게 되나요? 최고 속도와 평균 속도가 궁금합니다.
Output:

-------------------------------------------------------------------------------

In [32]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 12343
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n{instruction}\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


지구와의 비교점과 차이점에 대해 자세히 알려주세요.

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
지구와 달의 비교점과 차이점에 대해 자세히 알려주세요.
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
지구와 달은 우주에서 가까운 위치에 있음에도 불구하고, 그들 사이에는 많은 차이점이 존재합니다. 여러 측면에서 살펴봤을 때 지구와 달의 비교점과 차이점을 알려드리겠습니다.

 공통점
- 자전과 공전을 한다.
- 둘 모두 동그란 구 모양을 가지고 있다.
- 물이 존재한다.
- 태양계에 속해 있다.

 차이점
- 지구는 대기가 있지만, 달은 없다.
    - 이로 인해 지구는 일교차가 적고, 생물이 살 수 있지만, 달은 일교차가 크고 생물이 살 수 없다.
- 지구는 행성이지만, 달은 위성이다.
- 지구는 중력이 강하지만, 달은 중력이 약하다.
- 지구는 구름이 많지만, 달은 구름이 없다.
- 달은 크레이터가 존재하지만, 지구는 없다.
- 지구는 자전 주기가 더 길고, 달은 공전 주기가 더 길다.

이러한 지구와 달의 비교점과 차이점을 통해 두 행성이 어떤 측면에서 서로 다르고, 어떤 측면에서 유사한지를 이해할 수 있습니다.

---------------------------------------------------------------------------------------------------
PEFT MODEL:
지구와의 비교점과 차이점에 대해 자세히 알려주세요.

CPU times: user 25 s, sys: 245 ms, total: 25.2 s
Wall time: 25.3 s


In [33]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 11
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n{instruction}\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


극장elb 그냥 자랑elb 그냥 자랑elb 그냥 자랑elb 그냥 자랑elb 그냥 자랑elelb 그냥 자랑elb 그냥 자랑elb 그냥 자랑elelb 그냥 자랑elb 그냥 자랑elelb 그냥 자랑elb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥 자랑elelelb 그냥
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
극장 의자 밑에는 왜 구멍이 있을까요?
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
극장 

In [34]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 100
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n한국에 대해 설명해줘nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


한국에el 그냥 자랑ruction:Bel 그냥 자랑ruction' "'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
한국에 대해 설명해줘nOutput:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
꼽다와 꽂다는 많은 사람들이 헷갈려하는 단어입니다. 하지만 이 둘을 구분하는 방법이 있습니다. "꼽다"는 수나 날짜 등을 세려고 손가락을 헤아릴 때 사용하고, "꽂다"는 쓰러지지 않게 세우거나 고정할 때 사용합니다. 따라서 충전기를 콘센트에 연결할 때는 "꽂다"라는 표현을 사용하면 됩니다.

---------------------------------------------------------------------------------------------------
PEFT MODEL:
한국에el 그냥 자랑ruction:Bel 그냥 자랑ruction' "'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m"'8:m

In [35]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 100
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n한국을 빛낸 100명의 위인들nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:
한국을60struction:

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
한국을 빛낸 100명의 위인들nOutput:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
꼽다와 꽂다는 많은 사람들이 헷갈려하는 단어입니다. 하지만 이 둘을 구분하는 방법이 있습니다. "꼽다"는 수나 날짜 등

In [36]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 100
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n아름다운 학교nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
# print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
아름다운 학교nOutput:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
꼽다와 꽂다는 많은 사람들이 헷갈려하는 단어입니다. 하지만 이 둘을 구분하는 방법이 있습니다. "꼽다"는 수나 날짜 등을 세려고 손가락을 헤아릴 때 사용하고, "꽂다"는 쓰러지지 않게 세우거나 고정할 때 사용합니다. 따라서 충전기를 콘센트에 연결할 때는 "꽂다"라는 표현을 사용하면 됩니다.

---------------------------------------------------------------------------------------------------
PEFT MODEL:
belowruction 그냥bseostruction 그냥bseostructionelowruction 그냥bseostructionelowruction 그냥bseostructionelowruction 그냥bseostructionelowruction 그냥bseostructionelowruction 그냥bseostructionelowruction 그냥bseostructionelowruction 그냥bseostructionelowructionelowructionelowruction 그냥bseostructionelowruction 그냥bseostructionelowructionelowructionelowructionelowruction 그냥bseostructionelowructionelowructionelowructionelowructionelowructionelowructionelowructionelowructio

In [37]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 101
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction:\n{instruction}nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


전남 사찰에서 처음 본 꽃가루 모양의 벌레가 있어서 궁금합니다. 아에서 보듯이 꽃가루 모양의 벌레가 있었는데 처음 때 벌레임을 알 되었습니다. 이 벌레는 어떤 종류이며 외래종인가요?
전남 사찰에서 처음 본 꽃가루 모양의 벌레가 있었는데 처음 때 벌레임을 알 되었습니다. 이 벌레는 어떤 종류이며 외래종인가요?
전남 사찰에서 처음 본 꽃가루 모양의 벌레가 있었는데 처음 때 벌레임을 알 되었습니다. 이 벌레는 어떤 종류이며 외래종인가요?
전남 사찰에서 처음 본 꽃가루 모양의 벌레가 있었는데 처음 때 벌레임을 알 되었습니다. 이 벌레는 어떤 종류이며 외래종인가요?
전남 사찰에서 처음 본 꽃가루 모양의 벌레가 있었는데 처음 때 벌레임을 알 되었습니다. 이 벌레는 어떤 종류이며 외래종인가요?
전남 사찰에서 처음 본 꽃가루 모양의 벌레가 있었는데 처음 때 벌레임을 알 되었습니다. 이 벌레는 어떤 종류이며 외래종인가요?
전남 사찰에서 처음 본 꽃가루 모양의 벌레가 있었는데 처음 때 벌레임을 알 되었습니다. 이 벌레는 어떤 종류이며 외래종인가요
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
전남 사찰에서 발견한 꽃가루 모양의 벌레는 무엇인가요? 외래종인가요?
전남 사찰에서 처음 본 꽃가루 모양의 벌레가 있어서 궁금합니다. 사진에서 보듯이 꽃가루 같은 외형을 가지고 있었는데 움직일 때 벌레임을 알게 되었습니다. 이 벌레는 어떤 종류이며 외래종인가요?nOutput:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
사진에서 보이는 곤충은 매미목>선녀벌레과>갈색날개매미충(학명:Pochazia sp.) 입니다. 이 벌레는 최근에 우리나

In [39]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 101
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruction: Answer the questions below\n{instruction}\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction: Answer the questions below
전남 사찰에서 발견한 꽃가루 모양의 벌레는 무엇인가요? 외래종인가요?
전남 사찰에서 처음 본 꽃가루 모양의 벌레가 있어서 궁금합니다. 사진에서 보듯이 꽃가루 같은 외형을 가지고 있었는데 움직일 때 벌레임을 알게 되었습니다. 이 벌레는 어떤 종류이며 외래종인가요?
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
사진에서 보이는 곤충은 매미목>선녀벌레과>갈색날개매미충(학명:Pochazia sp.) 입니다. 이 벌레는 최근에 우리나라에 급속도로 번식하며 외래곤충으로 분류됩니다. 약충은 밝은 노란색 또는 흰색을 띠며, 몸의 끝부분에 돌기 모양의 털에 밀랍을 분비하여 꽃처럼 보이는 형태를 띱니다. 이러한 외형은 천적들의 시각적인 효과를 유도하여 무심결에 지나치도록 유도하는 것이며, 6~8월 사이에 갈색날매미충의 성충이나 약충은 식물의 즙액을 빨아먹으며 그을음병

In [42]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 999
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruct: Answer the questions below\n{instruction}\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Answer the questions below
보이차를 어떻게 마셔야 할까요? 처음에 첫물을 버려야 하나요? 보이차의 독성이 있을까요? 또 다른 음용법은 어떤 것이 있나요? 
어머니께서 유명다과점에서 구매한 보이차를 마셔보려고 하지만 어떻게 마셔야 할지 모르겠습니다. 처음 우린 물은 버려야 하는지, 보이차에 독성이 있는지 궁금합니다. 또한, 보이차를 마시는 다른 방법에 대해서도 알려주세요.
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
보이차는 제조 과정에서 독성을 제거하기 때문에 보통 첫물을 버리지 않아도 괜찮습니다. 하지만, 보이차의 첫물은 독성성분이 조금 높은 편이므로 차를 더욱 깨끗하게 즐기기 위해서는 버리는 것이 좋습니다. 보이차는 여러 가지 방법으로 음용할 수 있습니다. 일반적으로는 물이 팔팔 끓을 때 보이차를 넣어서 5분 정도 우려냅니다. 또 다른 방법은 다구를 이용하는 것인데, 보이차를 다구에 넣고 끓는 물을 부어서 20~30번 가량 우려내면 됩니다. 그리고 잘 우린 보이차는 거름막을 이용하여 걸러서 유리 주전자에 담아 식기 전에 마시면 됩니다.

---------------------------------------------------------------------------------------------------
PEFT MODEL:
어머니께서 유명다과점에서 구매한 보이차를적��셔야 될 있 처음에 첫물을 버려야 하나요? 보이차를적��셔야 될 있 처음에 첫물을 버려야 하나요? 보이차를적��셔야 될 있 처음에 첫물을 버려야 하나요?

In [43]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 999
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruct: Answer the questions below\n{instruction}\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Answer the questions below
보이차를 어떻게 마셔야 할까요? 처음에 첫물을 버려야 하나요? 보이차의 독성이 있을까요? 또 다른 음용법은 어떤 것이 있나요? 
어머니께서 유명다과점에서 구매한 보이차를 마셔보려고 하지만 어떻게 마셔야 할지 모르겠습니다. 처음 우린 물은 버려야 하는지, 보이차에 독성이 있는지 궁금합니다. 또한, 보이차를 마시는 다른 방법에 대해서도 알려주세요.
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
보이차는 제조 과정에서 독성을 제거하기 때문에 보통 첫물을 버리지 않아도 괜찮습니다. 하지만, 보이차의 첫물은 독성성분이 조금 높은 편이므로 차를 더욱 깨끗하게 즐기기 위해서는 버리는 것이 좋습니다. 보이차는 여러 가지 방법으로 음용할 수 있습니다. 일반적으로는 물이 팔팔 끓을 때 보이차를 넣어서 5분 정도 우려냅니다. 또 다른 방법은 다구를 이용하는 것인데, 보이차를 다구에 넣고 끓는 물을 부어서 20~30번 가량 우려내면 됩니다. 그리고 잘 우린 보이차는 거름막을 이용하여 걸러서 유리 주전자에 담아 식기 전에 마시면 됩니다.

---------------------------------------------------------------------------------------------------
PEFT MODEL:
어머니께서 유명다과점에서 구매한 보이차를적��셔야 될 있 처음에 첫물을 버려야 하나요? 보이차를적��셔야 될 있 처음에 첫물을 버려야 하나요? 보이차를적��셔야 될 있 처음에 첫물을 버려야 하나요?

In [44]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 98
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruct: Answer the questions below\n{instruction}\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Answer the questions below
포석정에서 술잔을 띄울 때 물 결의 흐름에 따라 거칠면 쏟아지지 않을까요? 
저는 포석정에서 술잔을 띄울 때 술잔이 왜 안 넘어지는지 궁금합니다. 파인 구멍으로 물이 흐르는데 흐름이 세질수록 술잔이 흔들리는데 쏟아질까 걱정입니다.
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
포석정의 수로는 와류 현상을 이용하여 술잔이 물결에 따라 쏟아지지 않도록 설계되어 있습니다. 물속도가 느리기 때문에 물결이 세지 않으며, 수로 굴곡에 의해 물이 일정한 장소에서 회돌이현상이 발생하여 술잔이 일정한 위치에서 머무르게 됩니다. 포석정의 수로 길이가 짧아 2-3분 걸리므로, 술잔을 띄운 후 셧구를 지을 시간으로 이용하던 것입니다. 이러한 과학 기술을 활용해 선조들은 유래 모양을 따른 수로를 만들고, 높이 약 20cm, 폭이 15cm 정도 되는 측벽을 63개의 석재로 안정적인 구조를 만들어 술잔을 안전하게 띄웠습니다. 우주선과 미사일 탱크 등 실무 공학적인 면에서는 회돌이 현상을 방지하나, 선조들은 와류 현상을 실용적으로 활용한 업적입니다.

---------------------------------------------------------------------------------------------------
PEFT MODEL:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
Out:
O

In [45]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 98
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\nInstruct: Answer the questions below\n\n{instruction}\nOutput:\n\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Below is an instruction that describes a task. Write a response that appropriately completes the request.
Instruct: Answer the questions below

포석정에서 술잔을 띄울 때 물 결의 흐름에 따라 거칠면 쏟아지지 않을까요? 
저는 포석정에서 술잔을 띄울 때 술잔이 왜 안 넘어지는지 궁금합니다. 파인 구멍으로 물이 흐르는데 흐름이 세질수록 술잔이 흔들리는데 쏟아질까 걱정입니다.
Output:


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
포석정의 수로는 와류 현상을 이용하여 술잔이 물결에 따라 쏟아지지 않도록 설계되어 있습니다. 물속도가 느리기 때문에 물결이 세지 않으며, 수로 굴곡에 의해 물이 일정한 장소에서 회돌이현상이 발생하여 술잔이 일정한 위치에서 머무르게 됩니다. 포석정의 수로 길이가 짧아 2-3분 걸리므로, 술잔을 띄운 후 셧구를 지을 시간으로 이용하던 것입니다. 이러한 과학 기술을 활용해 선조들은 유래 모양을 따른 수로를 만들고, 높이 약 20cm, 폭이 15cm 정도 되는 측벽을 63개의 석재로 안정적인 구조를 만들어 술잔을 안전하게 띄웠습니다. 우주선과 미사일 탱크 등 실무 공학적인 면에서는 회돌이 현상을 방지하나, 선조들은 와류 현상을 실용적으로 활용한 업적입니다.

---------------------------------------------------------------------------------------------------
PEFT MODEL

In [47]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 98
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruct: Answer the questions below\n\n{instruction}\n\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Answer the questions below

포석정에서 술잔을 띄울 때 물 결의 흐름에 따라 거칠면 쏟아지지 않을까요? 
저는 포석정에서 술잔을 띄울 때 술잔이 왜 안 넘어지는지 궁금합니다. 파인 구멍으로 물이 흐르는데 흐름이 세질수록 술잔이 흔들리는데 쏟아질까 걱정입니다.

Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
포석정의 수로는 와류 현상을 이용하여 술잔이 물결에 따라 쏟아지지 않도록 설계되어 있습니다. 물속도가 느리기 때문에 물결이 세지 않으며, 수로 굴곡에 의해 물이 일정한 장소에서 회돌이현상이 발생하여 술잔이 일정한 위치에서 머무르게 됩니다. 포석정의 수로 길이가 짧아 2-3분 걸리므로, 술잔을 띄운 후 셧구를 지을 시간으로 이용하던 것입니다. 이러한 과학 기술을 활용해 선조들은 유래 모양을 따른 수로를 만들고, 높이 약 20cm, 폭이 15cm 정도 되는 측벽을 63개의 석재로 안정적인 구조를 만들어 술잔을 안전하게 띄웠습니다. 우주선과 미사일 탱크 등 실무 공학적인 면에서는 회돌이 현상을 방지하나, 선조들은 와류 현상을 실용적으로 활용한 업적입니다.

---------------------------------------------------------------------------------------------------
PEFT MODEL:
포석정에서 술잔을 띄울 때 물 결의 어려웠에 따라 거칠면 쏟아지지 않을까요?

포석정에서 술잔을 띄울 때 물 결의 어려웠에 따라 거칠면 쏟아지지 않을까요?

포석정에서 술잔을 띄울 때 물

In [49]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 121
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Instruct: Answer the questions below\n\n{instruction}\n\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Answer the questions below

미국에도 동시지방선거가 있나요?

Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
미국은 일년 내내 다양한 선거를 진행합니다. 대통령과 주지사, 시장 등 일부 고위직은 4년 주기로 교체되며, 연방 하원의회의 경우 2년마다 1/3가 변경됩니다. 또한 교육위원, 선거관리위원, 시의회 의원, 지역 법원판사 등 많은 지방공직자들이 선출됩니다. 토지평가 업무를 담당하는 직책과 세금징수 업무를 담당하는 공무원들도 선출직입니다. 미국에서는 도시의 크기나 구성원 수에 따라 다양한 유형의 선거가 실시되며, 그만큼 많은 수의 선거인들이 길거리에 등록되어 있어 일년 내내 투표를 할 수 있습니다. 

단순한 동시지방선거와는 달리, 미국에서는 다양한 종류의 지방선거가 있으며, 한 해 내내 선거가 이루어집니다. 따라서 미국에서는 일반적인 '동시지방선거'는 없다고 볼 수 있습니다.

---------------------------------------------------------------------------------------------------
PEFT MODEL:
#: Anser the questions below

미국에도 동시지방선거가 있나요?


CPU times: user 24.2 s, sys: 132 ms, total: 24.3 s
Wall time: 24.4 s


In [50]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 121
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\nInstruct: Answer the questions below\n\n{instruction}\n\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Below is an instruction that describes a task. Write a response that appropriately completes the request.
Instruct: Answer the questions below

미국에도 동시지방선거가 있나요?

Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
미국은 일년 내내 다양한 선거를 진행합니다. 대통령과 주지사, 시장 등 일부 고위직은 4년 주기로 교체되며, 연방 하원의회의 경우 2년마다 1/3가 변경됩니다. 또한 교육위원, 선거관리위원, 시의회 의원, 지역 법원판사 등 많은 지방공직자들이 선출됩니다. 토지평가 업무를 담당하는 직책과 세금징수 업무를 담당하는 공무원들도 선출직입니다. 미국에서는 도시의 크기나 구성원 수에 따라 다양한 유형의 선거가 실시되며, 그만큼 많은 수의 선거인들이 길거리에 등록되어 있어 일년 내내 투표를 할 수 있습니다. 

단순한 동시지방선거와는 달리, 미국에서는 다양한 종류의 지방선거가 있으며, 한 해 내내 선거가 이루어집니다. 따라서 미국에서는 일반적인 '동시지방선거'는 없다고 볼 수 있습니다.

---------------------------------------------------------------------------------------------------
PEFT MODEL:
미국에도 동시지방선거가 있나요?

미국에도 동시지방선거가 있나요?

미국에도 동시지방선거가 있나요?

미국에도 동시지방선거가 있나요?

미국에도 동시지방선거가 있나요?



In [51]:
%%time
## 인간 평가 방법
from transformers import set_seed
seed = 42

set_seed(seed)

index = 2222
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']

prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\nInstruct: Answer the questions below\n\n{instruction}\n\nOutput:\n"

#원래 모델을 사용하여 수행한 것처럼 동일한 입력을 사용하지만 PEFT 모델을 사용하여 추론을 수행
peft_model_res = gen(ft_model,prompt,350)
peft_model_output = peft_model_res.split('Output:\n')[1]
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Below is an instruction that describes a task. Write a response that appropriately completes the request.
Instruct: Answer the questions below

이준 열사는 자살로 죽은 건가요? 
제 학원 사회 선생님의 말씀에 의하면 이준 열사는 만국 평화회의 참석을 못하고 심한 감기로 병이 심해져 죽었다고 합니다. 특사들은 돈이 없어 약을 제대로 쓰지 못했다는데, 이 말이 사실인가요?

Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
이준 열사는 감기 앓기는 했지만, 자살로 죽은 것은 아닙니다. 당시에는 만국 평화회의 안보협의회에 참석하기 위해 네덜란드로 출국한 이준 열사가 기차 여행을 하면서 육체적, 정신적 지침과 질병에 걸려 병원에 입원했고, 최종적으로 1967년 3월 21일 오후 6시 50분 병원에서 세상을 떠났습니다. 병원 진단서는 아직 발견되지 않았지만, 이위종 밀사의 말에 따르면 이준 열사의 병은 종양이 악화되어 돌아간 것으로 추정됩니다. 따라서 이준 열사는 자살로 죽은 것은 아니며, 감기로 인한 악화로 인해 사망하였습니다. 만약, 어디선가 위와 같은 소문이 나와 다른 정보를 알리고 있다면, 그 대부분은 사실이 아닐 가능성이 높습니다.

---------------------------------------------------------------------------------------------------
PEFT MODEL:
이준 열사는 자살로 죽은 건가요?